In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import altair as alt
from sklearn import preprocessing


In [ ]:
# info on dataset from https://www.detroitk12.org/Page/9634

# Student assessment in grades 3 through 8 includes the Michigan Student Test of Educational Progress (M-STEP), a summative assessment based on Michigan’s content standards, designed to measure achievement and growth effectively for most students.

# Students are tested in all grades from 3 through 8 in Math and English language arts (ELA), grades 4 and 7 in Science, and grades 5 and 8 in Social Studies. For M-STEP, 11th grade students are tested in Science, and Social Studies. For MI-Access, 11th grade students are tested in ELA, Mathematics, Science, and Social Studies.

# MI-Access tests come in three varieties: Functional Independence (FI), Supported Independence (SI), and Participation (P). More information about the MI-Access test and test accommodations for students can be found at the MI-Access website.

# You can explore the M-STEP report by entity (school, district, intermediate school district, and statewide), by subject (ELA, math, science, or social studies), and by different demographic factors such as gender, ethnicity, and socioeconomic status.

# The M-STEP report shows student performance, scaled score, and proficiency in ELA, math, science, and social studies.

#Isd stands for intermediate school district


In [ ]:
#list of application schools in the district from https://www.detroitk12.org/Page/15142

# What are the K-12 application schools for the 2022-23 school year? 

# K-8 Application Schools 

# Academy of the Americas (K-8) 
# Bates Academy (K-8) 
# Clippert Multicultural Honors Academy (6-8) 
# Chrysler Elementary (K-5) 
# Detroit International Academy (K-8) 
# Foreign Language Immersion and Cultural Studies School (FLICS) (K-8) 
# Golightly Education Center (K-8) 
# Marcus Garvey Academy (K-8) 
# Paul Robeson/ Malcolm X Academy (K-8) 
# The School at Marygrove (K-2)

# High School Application Schools 

# Academy of the Americas 
# Benjamin Carson High School of Science and Medicine 
# Davis Aerospace Technical High School 
# Detroit International Academy for Young Women 
# Detroit School of Arts 
# Frederick Douglass Academy for Young Men 

In [ ]:
#import testing data
test = pd.read_csv('/work/SIADS697_TeamMADScientists/assets/testing_data/DPSCD_OpenData_MSTEP_Growth_2018-2019_20191101.csv')
test_count = test['BuildingName'].unique()

#dataset contains 95 of the 107 schools in the Detroit Public Schools Community District

In [ ]:
#import zip code data
zip_code = pd.read_csv('/work/SIADS697_TeamMADScientists/assets/testing_data/DPSCD_School_List_2018-2019_20190226.csv')
zip_count = zip_code['BuildingName'].unique()

In [ ]:
#finding the columns that the two datasets have in common so no info is repeated. 

same_name = []

for col in zip_code.columns:
    for cols in test.columns:
        if col == cols:
            same_name.append(col)

print(same_name)

['SchoolYear', 'BuildingCode', 'BuildingName', 'DistrictCode', 'DistrictName', 'EntityType']


In [ ]:
#dropping all duplicate columns except BuildingName which we will merge with
zip_code = zip_code.drop(['SchoolYear', 'BuildingCode', 'DistrictCode', 'DistrictName', 'EntityType', 'COUNTY_NAME', 'LOCALE_NAME', 'SCHOOL_LEVEL', 'SCHOOL_TYPE', 'SETTING'], axis = 1)
test = test.drop(['IsdCode','IsdName'], axis = 1)

In [ ]:
# merging the data sets
df_test = test.merge(zip_code, how='inner', on='BuildingName')
df_test.head()

,SchoolYear,DistrictCode,DistrictName,BuildingCode,BuildingName,CountyCode,CountyName,EntityType,SchoolLevel,Locale,...,MeanSGP,ISDCode,ISDName,Status,SCHOOL_EMPHASIS,PhoneNumber,ADDRESS_LINE_1,CITY,STATE,ZIP_CODE
0,2018-2019,82015,Detroit Public Schools Community District,4,Henderson Academy,82,Wayne,LEA School,Elem_Middle School,City: Large,...,< 10,82,Wayne RESA,Open-Active,General Education,3138520512,16101 West Chicago St,Detroit,MI,48228
1,2018-2019,82015,Detroit Public Schools Community District,4,Henderson Academy,82,Wayne,LEA School,Elem_Middle School,City: Large,...,< 10,82,Wayne RESA,Open-Active,General Education,3138520512,16101 West Chicago St,Detroit,MI,48228
2,2018-2019,82015,Detroit Public Schools Community District,4,Henderson Academy,82,Wayne,LEA School,Elem_Middle School,City: Large,...,25.80,82,Wayne RESA,Open-Active,General Education,3138520512,16101 West Chicago St,Detroit,MI,48228
3,2018-2019,82015,Detroit Public Schools Community District,4,Henderson Academy,82,Wayne,LEA School,Elem_Middle School,City: Large,...,25.80,82,Wayne RESA,Open-Active,General Education,3138520512,16101 West Chicago St,Detroit,MI,48228
4,2018-2019,82015,Detroit Public Schools Community District,4,Henderson Academy,82,Wayne,LEA School,Elem_Middle School,City: Large,...,25.60,82,Wayne RESA,Open-Active,General Education,3138520512,16101 West Chicago St,Detroit,MI,48228


In [ ]:
#There are no missing values in the dataframe
df_test.isnull().values.any()

False

In [ ]:
#exploring data, looking at how many schools have reported precentages and numbers and how much data there is for testing groups. 
df_test_eco_dis = df_test.loc[(df_test['TestingGroup'] == 'Economically Disadvantaged') & (df_test['Subject']  == 'Mathematics') & (df_test['Grade'] == 'All Grades')]


In [ ]:
_deepnote_run_altair(df_test_eco_dis, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":true},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"BuildingName","type":"nominal","sort":{"order":"descending","encoding":"y"},"scale":{"type":"linear","zero":false}},"y":{"field":"PercentAboveAverage","type":"quantitative","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"ZIP_CODE","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [ ]:
# checking to see why West Side did not show a bar on the graph
# its because it the value has a < symbol
df_west = df_test[df_test['BuildingName'].str.contains('West Side')]
df_west

,SchoolYear,DistrictCode,DistrictName,BuildingCode,BuildingName,CountyCode,CountyName,EntityType,SchoolLevel,Locale,...,MeanSGP,ISDCode,ISDName,Status,SCHOOL_EMPHASIS,PhoneNumber,ADDRESS_LINE_1,CITY,STATE,ZIP_CODE
11049,2018-2019,82015,Detroit Public Schools Community District,8929,West Side Academy of Information Technology an...,82,Wayne,LEA School,High School,City: Large,...,< 10,82,Wayne RESA,Open-Active,Alternative Education,3135960775,4701 Mckinley,Detroit,MI,48208
11050,2018-2019,82015,Detroit Public Schools Community District,8929,West Side Academy of Information Technology an...,82,Wayne,LEA School,High School,City: Large,...,< 10,82,Wayne RESA,Open-Active,Alternative Education,3135960775,4701 Mckinley,Detroit,MI,48208
11051,2018-2019,82015,Detroit Public Schools Community District,8929,West Side Academy of Information Technology an...,82,Wayne,LEA School,High School,City: Large,...,< 10,82,Wayne RESA,Open-Active,Alternative Education,3135960775,4701 Mckinley,Detroit,MI,48208
11052,2018-2019,82015,Detroit Public Schools Community District,8929,West Side Academy of Information Technology an...,82,Wayne,LEA School,High School,City: Large,...,< 10,82,Wayne RESA,Open-Active,Alternative Education,3135960775,4701 Mckinley,Detroit,MI,48208
11053,2018-2019,82015,Detroit Public Schools Community District,8929,West Side Academy of Information Technology an...,82,Wayne,LEA School,High School,City: Large,...,< 10,82,Wayne RESA,Open-Active,Alternative Education,3135960775,4701 Mckinley,Detroit,MI,48208
11054,2018-2019,82015,Detroit Public Schools Community District,8929,West Side Academy of Information Technology an...,82,Wayne,LEA School,High School,City: Large,...,< 10,82,Wayne RESA,Open-Active,Alternative Education,3135960775,4701 Mckinley,Detroit,MI,48208
11055,2018-2019,82015,Detroit Public Schools Community District,8929,West Side Academy of Information Technology an...,82,Wayne,LEA School,High School,City: Large,...,< 10,82,Wayne RESA,Open-Active,Alternative Education,3135960775,4701 Mckinley,Detroit,MI,48208
11056,2018-2019,82015,Detroit Public Schools Community District,8929,West Side Academy of Information Technology an...,82,Wayne,LEA School,High School,City: Large,...,< 10,82,Wayne RESA,Open-Active,Alternative Education,3135960775,4701 Mckinley,Detroit,MI,48208
11057,2018-2019,82015,Detroit Public Schools Community District,8929,West Side Academy of Information Technology an...,82,Wayne,LEA School,High School,City: Large,...,< 10,82,Wayne RESA,Open-Active,Alternative Education,3135960775,4701 Mckinley,Detroit,MI,48208
11058,2018-2019,82015,Detroit Public Schools Community District,8929,West Side Academy of Information Technology an...,82,Wayne,LEA School,High School,City: Large,...,< 10,82,Wayne RESA,Open-Active,Alternative Education,3135960775,4701 Mckinley,Detroit,MI,48208


In [ ]:
_deepnote_run_altair(df_test_eco_dis, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":true},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"BuildingName","type":"nominal","sort":{"order":"descending","encoding":"y"},"scale":{"type":"linear","zero":false}},"y":{"field":"MeanSGP","type":"nominal","sort":{"order":"ascending","encoding":"y"},"scale":{"type":"linear","zero":true}},"color":{"field":"ZIP_CODE","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [ ]:
_deepnote_run_altair(df_test_eco_dis, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"point","tooltip":true},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"BuildingName","type":"nominal","sort":{"order":"ascending","encoding":"y"},"scale":{"type":"linear","zero":false}},"y":{"field":"ZIP_CODE","type":"nominal","sort":{"order":"descending","encoding":"y"},"scale":{"type":"linear","zero":false}},"color":{"field":"","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

In [ ]:
# Finding which schools are magnet schools within the district

df_magnet_schools = df_test[df_test['BuildingName'].str.contains("Magnet")]
df_magnet_schools['BuildingName'].unique()

array(['Ann Arbor Trail Magnet School', 'Ludington Magnet Middle School',
       'Fisher Magnet Lower Academy', 'Fisher Magnet Upper Academy'],
      dtype=object)

In [ ]:
_deepnote_run_altair(df_magnet_schools, """{"$schema":"https://vega.github.io/schema/vega-lite/v4.json","mark":{"type":"bar","tooltip":true},"height":220,"autosize":{"type":"fit"},"data":{"name":"placeholder"},"encoding":{"x":{"field":"BuildingName","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}},"y":{"field":"PercentAverageGrowth","type":"quantitative","sort":null,"scale":{"type":"linear","zero":true}},"color":{"field":"ZIP_CODE","type":"nominal","sort":null,"scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

## Descriptive Incites

Creating a data frame with quantitative data 

In [ ]:
#checking what kinds of values are in each column
df_test.dtypes
#Looking at the unique values in each column I want to chang to float

# df_test['NumberAboveAverageGrowth'].unique()
# df_test['NumberAverageGrowth'].unique()
# df_test['NumberBelowAverageGrowth'].unique()
# df_test['PercentAboveAverage'].unique()
# df_test['PercentAverageGrowth'].unique()
# df_test['PercentBelowAverage'].unique()
# df_test['TotalIncluded'].unique()
# df_test['MeanSGP'].unique()

df_test_int = df_test.copy()

#changing values with < or > to just the number in order to plot easier
df_test_int['NumberAboveAverageGrowth'] = df_test_int['NumberAboveAverageGrowth'].replace('< 10', 10)
df_test_int['NumberAverageGrowth'] = df_test_int['NumberAverageGrowth'].replace('< 10', 10)
df_test_int['NumberBelowAverageGrowth'] = df_test_int['NumberBelowAverageGrowth'].replace('< 10', 10)
df_test_int['PercentAboveAverage'] = df_test_int['PercentAboveAverage'].replace('< 10', 10)
df_test_int['PercentAboveAverage'] = df_test_int['PercentAboveAverage'].replace('< 5', 5)
df_test_int['PercentAverageGrowth'] = df_test_int['PercentAverageGrowth'].replace('< 10', 10)
df_test_int['PercentAverageGrowth'] = df_test_int['PercentAverageGrowth'].replace('< 5', 5)
df_test_int['PercentBelowAverage'] = df_test_int['PercentBelowAverage'].replace('< 10', 10)
df_test_int['PercentBelowAverage'] = df_test_int['PercentBelowAverage'].replace('< 5', 5)
df_test_int['TotalIncluded'] = df_test_int['TotalIncluded'].replace('< 10', 10)
df_test_int['MeanSGP'] = df_test_int['MeanSGP'].replace('< 10', 10)


normalizing the quantitative data

In [ ]:
#normalizing all of the quantitative data because the number of individuals for different groups differ so much
col = ['NumberAboveAverageGrowth', 'NumberAverageGrowth', 'NumberBelowAverageGrowth', 'PercentAboveAverage', 'PercentAverageGrowth', 'PercentBelowAverage', 'TotalIncluded', 'MeanSGP' ]

#turning objects into floats
for col_name in col:
    df_test_int[col_name] =  df_test_int[col_name].astype(float)

new_cols = []
for col_name in col:
    x = np.array(df_test_int[col_name])
    norm = preprocessing.normalize([x])
    new_cols.append(norm)

In [ ]:
new_df = df_test_int[['SchoolYear','BuildingName','CountyName', 'SchoolLevel',
       'Locale', 'Grade', 'Subject', 'TestingGroup', 'ISDCode', 'ISDName', 'Status', 'SCHOOL_EMPHASIS',
       'PhoneNumber', 'ADDRESS_LINE_1', 'CITY', 'STATE', 'ZIP_CODE']]

NumberAboveAverageGrowth = new_cols[0][0].tolist()
NumberAverageGrowth = new_cols[1][0].tolist()
NumberBelowAverageGrowth = new_cols[2][0].tolist()
PercentAboveAverage = new_cols[3][0].tolist()
PercentAverageGrowth = new_cols[4][0].tolist()
PercentBelowAverage = new_cols[5][0].tolist()
TotalIncluded =  new_cols[6][0].tolist()
MeanSGP = new_cols[7][0].tolist()

new_df['NumberAboveAverageGrowth'] = NumberAboveAverageGrowth
new_df['NumberAverageGrowth'] = NumberAverageGrowth
new_df['NumberBelowAverageGrowth'] = NumberBelowAverageGrowth
new_df['PercentAboveAverage'] = PercentAboveAverage
new_df['PercentAverageGrowth'] = PercentAverageGrowth
new_df['PercentBelowAverage'] = PercentBelowAverage
new_df['TotalIncluded'] = TotalIncluded
new_df['MeanSGP'] = MeanSGP

/tmp/ipykernel_1082/148430513.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['NumberAboveAverageGrowth'] = NumberAboveAverageGrowth
/tmp/ipykernel_1082/148430513.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['NumberAverageGrowth'] = NumberAverageGrowth
/tmp/ipykernel_1082/148430513.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

In [ ]:
#looking at the different types of groups in order to compare ones of interest
df_test_int['TestingGroup'].unique()


array(['Students with Disabilities', 'Students without Disabilities',
       'All Students', 'Economically Disadvantaged',
       'Not Economically Disadvantaged',
       'American Indian or Alaska Native',
       'Black, not of Hispanic origin', 'Male', 'Female',
       'Not English Learners', 'Not Migrant',
       'White, not of Hispanic origin', 'Two or More Races',
       'English Learners', 'Hispanic', 'Asian',
       'Native Hawaiian or Other Pacific Islander'], dtype=object)

In [ ]:
df_percabove_disability = new_df.loc[((new_df['TestingGroup'] == 'Students with Disabilities') | (df_test_int['TestingGroup'] =='Students without Disabilities')) & (df_test_int['Grade'] == 'All Grades') & (df_test_int['Subject'] == 'Mathematics')]

In [ ]:
alt.Chart(df_percabove_disability).mark_circle(size=60).encode(
    x='BuildingName',
    y='PercentAboveAverage',
    color='TestingGroup',
    tooltip=['BuildingName', 'PercentAboveAverage', 'TestingGroup']
).interactive()

alt.Chart(...)

## Disabled Students

Comparing the avg.  number of students with disabilities above avg, below avg and at avg. per zip code 

In [ ]:
df_zip_dis = new_df.loc[(new_df['TestingGroup'] == 'Students with Disabilities') & (df_test_int['Grade'] == 'All Grades')]
# df_zip_dis['TestingGroup'].unique()
df_zip_dis = df_zip_dis.groupby(['ZIP_CODE']).mean()
df_zip_dis = df_zip_dis.reset_index()
df_zip_dis = df_zip_dis.sort_values('NumberAboveAverageGrowth', axis = 0, ascending = True)
df_zip_dis.head()

,ZIP_CODE,ISDCode,PhoneNumber,NumberAboveAverageGrowth,NumberAverageGrowth,NumberBelowAverageGrowth,PercentAboveAverage,PercentAverageGrowth,PercentBelowAverage,TotalIncluded,MeanSGP
22,48234,82.0,3.138665e+09,0.003726,0.002699,0.003273,0.005573,0.007358,0.007800,0.001853,0.006262
11,48213,82.0,3.138624e+09,0.003726,0.002993,0.003057,0.006025,0.006392,0.008394,0.002020,0.006179
0,48201,82.0,3.134365e+09,0.003726,0.002672,0.002727,0.004002,0.003931,0.004285,0.001239,0.003386
23,48235,82.0,3.136135e+09,0.003726,0.002721,0.003049,0.005352,0.004566,0.006313,0.001580,0.004743
16,48219,82.0,3.135973e+09,0.003726,0.003073,0.003864,0.004282,0.007218,0.008486,0.002280,0.005398


In [ ]:
all_dis = alt.Chart(df_zip_dis).transform_fold(
    ['NumberAboveAverageGrowth', 'NumberAverageGrowth', 'NumberBelowAverageGrowth'],
).mark_bar().encode(
    x='ZIP_CODE:N',
    y='value:Q',
    color='key:N',
    tooltip = ['ZIP_CODE','NumberAboveAverageGrowth', 'NumberAverageGrowth', 'NumberBelowAverageGrowth']
).interactive(
).properties(
        title = 'Average Number Above, At and Below Average Growth For Disabled Students per Zip Code'
)
all_dis

alt.Chart(...)

Comparing the percent above, at and below average for disabled students per zip code.

In [ ]:
Perc_dis = alt.Chart(df_zip_dis).transform_fold(
    ['PercentAboveAverage', 'PercentAverageGrowth', 'PercentBelowAverage'],
).mark_bar().encode(
    x='ZIP_CODE:N',
    y='value:Q',
    color='key:N',
    tooltip = ['ZIP_CODE','PercentAboveAverage', 'PercentAverageGrowth', 'PercentBelowAverage']
).interactive(
).properties(
        title = 'Average Percent Above, At and Below Average Growth For Disabled Students per Zip Code'
)
Perc_dis

alt.Chart(...)

In [ ]:
#creating dataframe with diabled and abled indivudials to compare
df_zip_dis_nondis = new_df.loc[((new_df['TestingGroup'] == 'Students with Disabilities') | (df_test_int['TestingGroup'] == 'Students without Disabilities')) & (df_test_int['Grade'] == 'All Grades')]
# df_zip_dis['TestingGroup'].unique()
df_zip_dis_nondis = df_zip_dis_nondis.groupby(['ZIP_CODE', 'TestingGroup']).mean()
df_zip_dis_nondis = df_zip_dis_nondis.reset_index()
#df_zip_dis_nondis = df_zip_dis_nondis.sort_values('NumberAboveAverageGrowth', axis = 0, ascending = True)
df_zip_dis_nondis.head()


,ZIP_CODE,TestingGroup,ISDCode,PhoneNumber,NumberAboveAverageGrowth,NumberAverageGrowth,NumberBelowAverageGrowth,PercentAboveAverage,PercentAverageGrowth,PercentBelowAverage,TotalIncluded,MeanSGP
0,48201,Students with Disabilities,82.0,3.134365e+09,0.003726,0.002672,0.002727,0.004002,0.003931,0.004285,0.001239,0.003386
1,48201,Students without Disabilities,82.0,3.134365e+09,0.015400,0.019572,0.019115,0.007117,0.010638,0.010873,0.019330,0.009481
2,48202,Students with Disabilities,82.0,3.136838e+09,0.003726,0.002717,0.002840,0.006304,0.007248,0.008596,0.001797,0.007097
3,48202,Students without Disabilities,82.0,3.136838e+09,0.008259,0.007348,0.006672,0.011743,0.009631,0.008639,0.007362,0.011453
4,48203,Students with Disabilities,82.0,3.138665e+09,0.003726,0.002672,0.002937,0.007662,0.007701,0.008095,0.001794,0.007976


In [ ]:
#Chart showing the difference between students with and without disabilities who were above average
zip_dis_nondis_numabove = alt.Chart(df_zip_dis_nondis).mark_bar().encode(
    alt.X('ZIP_CODE:N', sort=alt.EncodingSortField(field="ZIP_CODE", op="count", order='ascending')),
    y=alt.Y('PercentAboveAverage:Q'),
    color = 'TestingGroup:N',
    tooltip=['ZIP_CODE', 'TestingGroup','PercentAboveAverage']
).interactive(
).properties(
        title = 'Average Percent of Disabled vs. Non-disabled Students Above Average Growth per Zip Code'
)

zip_dis_nondis_numabove

alt.Chart(...)

In [ ]:
#Chart showing the difference between students with and without disabilities who were above average
perc_above_avg_dis = alt.Chart(df_zip_dis_nondis).mark_bar().encode(
    x=alt.X('TestingGroup:N', title = None),
    y='PercentAboveAverage:Q',
    color='TestingGroup:N',
    column='ZIP_CODE:N'
)
perc_above_avg_dis

alt.Chart(...)

In [ ]:
#Chart showing the difference between students with and without disabilities who were at average
zip_dis_nondis_numat = alt.Chart(df_zip_dis_nondis).mark_bar().encode(
    alt.X('ZIP_CODE:N', sort=alt.EncodingSortField(field="ZIP_CODE", op="count", order='ascending')),
    y=alt.Y('PercentAverageGrowth:Q'),
    color = 'TestingGroup:N',
    tooltip=['ZIP_CODE', 'TestingGroup','PercentAverageGrowth']
).interactive(
).properties(
        title = 'Average Percent of Disabled vs. Non-disabled Students At Average Growth per Zip Code'
)

zip_dis_nondis_numat

alt.Chart(...)

In [ ]:
#Chart showing the difference between students with and without disabilities who were at average
perc_at_avg_dis = alt.Chart(df_zip_dis_nondis).mark_bar().encode(
    x=alt.X('TestingGroup:N', title = None),
    y='PercentAverageGrowth:Q',
    color='TestingGroup:N',
    column='ZIP_CODE:N'
)

perc_at_avg_dis

alt.Chart(...)

In [ ]:
#Chart showing the difference between students with and without disabilities who were below average
zip_dis_nondis_numat = alt.Chart(df_zip_dis_nondis).mark_bar().encode(
    alt.X('ZIP_CODE:N', sort=alt.EncodingSortField(field="ZIP_CODE", op="count", order='ascending')),
    y=alt.Y('PercentBelowAverage:Q'),
    color = 'TestingGroup:N',
    tooltip=['ZIP_CODE', 'TestingGroup','PercentBelowAverage']
).interactive(
).properties(
        title = 'Average Percent of Disabled vs. Non-disabled Students Below Average Growth per Zip Code'
)

zip_dis_nondis_numat

alt.Chart(...)

In [ ]:
#Chart showing the difference between students with and without disabilities who were below average
perc_below_avg_dis = alt.Chart(df_zip_dis_nondis).mark_bar().encode(
    x=alt.X('TestingGroup:N', title = None),
    y='PercentBelowAverage:Q',
    color='TestingGroup:N',
    column='ZIP_CODE:N'
)
perc_below_avg_dis

alt.Chart(...)

## Economically Disadvantaged

Comparing the avg.  number of students with disabilities above avg, below avg and at avg. per zip code 

In [ ]:
df_zip_ecodis = new_df.loc[(new_df['TestingGroup'] == 'Economically Disadvantaged') & (df_test_int['Grade'] == 'All Grades')]
df_zip_ecodis = df_zip_ecodis.groupby(['ZIP_CODE']).mean()
df_zip_ecodis = df_zip_ecodis.reset_index()
df_zip_ecodis = df_zip_ecodis.sort_values('NumberAboveAverageGrowth', axis = 0, ascending = True)
df_zip_ecodis.head()

,ZIP_CODE,ISDCode,PhoneNumber,NumberAboveAverageGrowth,NumberAverageGrowth,NumberBelowAverageGrowth,PercentAboveAverage,PercentAverageGrowth,PercentBelowAverage,TotalIncluded,MeanSGP
26,48239,82.0,3.132749e+09,0.005216,0.006947,0.011271,0.006379,0.009737,0.012229,0.007990,0.008479
15,48217,82.0,3.133866e+09,0.005837,0.004631,0.005139,0.009693,0.008169,0.011390,0.004815,0.010039
12,48214,82.0,3.138616e+09,0.007517,0.006947,0.006571,0.007832,0.009840,0.009082,0.006705,0.009356
22,48234,82.0,3.138665e+09,0.007638,0.008470,0.011117,0.006025,0.009363,0.011017,0.009274,0.007977
1,48202,82.0,3.136838e+09,0.008135,0.007838,0.006853,0.010341,0.010136,0.009127,0.007763,0.010958


In [ ]:
all_ecodis = alt.Chart(df_zip_ecodis).transform_fold(
    ['NumberAboveAverageGrowth', 'NumberAverageGrowth', 'NumberBelowAverageGrowth'],
).mark_bar().encode(
    x='ZIP_CODE:N',
    y='value:Q',
    color='key:N',
    tooltip = ['ZIP_CODE','NumberAboveAverageGrowth', 'NumberAverageGrowth', 'NumberBelowAverageGrowth']
).interactive(
).properties(
        title = 'Average Number Above, At and Below Average Growth For Economically Disadvantage Students per Zip Code'
)
all_ecodis

alt.Chart(...)

In [ ]:
perc_ecodis = alt.Chart(df_zip_ecodis).transform_fold(
    ['PercentAboveAverage', 'PercentAverageGrowth', 'PercentBelowAverage'],
).mark_bar().encode(
    x='ZIP_CODE:N',
    y='value:Q',
    color='key:N',
    tooltip = ['ZIP_CODE','PercentAboveAverage', 'PercentAverageGrowth', 'PercentBelowAverage']
).interactive(
).properties(
        title = 'Average Percent Above, At and Below Average Growth For Economically Disadvantages Students per Zip Code'
)
perc_ecodis

alt.Chart(...)

In [ ]:
#creating dataframe with diabled and abled indivudials to compare
df_zip_ecodis_not = new_df.loc[((new_df['TestingGroup'] == 'Economically Disadvantaged') | (df_test_int['TestingGroup'] == 'Not Economically Disadvantaged')) & (df_test_int['Grade'] == 'All Grades')]
# df_zip_dis['TestingGroup'].unique()
df_zip_ecodis_not = df_zip_ecodis_not.groupby(['ZIP_CODE', 'TestingGroup']).mean()
df_zip_ecodis_not = df_zip_ecodis_not.reset_index()
#df_zip_dis_nondis = df_zip_dis_nondis.sort_values('NumberAboveAverageGrowth', axis = 0, ascending = True)
df_zip_ecodis_not.head()

,ZIP_CODE,TestingGroup,ISDCode,PhoneNumber,NumberAboveAverageGrowth,NumberAverageGrowth,NumberBelowAverageGrowth,PercentAboveAverage,PercentAverageGrowth,PercentBelowAverage,TotalIncluded,MeanSGP
0,48201,Economically Disadvantaged,82.0,3.134365e+09,0.012574,0.016032,0.015734,0.007120,0.010592,0.010916,0.015823,0.009491
1,48201,Not Economically Disadvantaged,82.0,3.134365e+09,0.005309,0.005121,0.004959,0.006388,0.008428,0.007437,0.004361,0.007530
2,48202,Economically Disadvantaged,82.0,3.136838e+09,0.008135,0.007838,0.006853,0.010341,0.010136,0.009127,0.007763,0.010958
3,48202,Not Economically Disadvantaged,82.0,3.136838e+09,0.003726,0.002672,0.002705,0.008030,0.005474,0.009281,0.001518,0.006889
4,48203,Economically Disadvantaged,82.0,3.138665e+09,0.011443,0.009466,0.010163,0.010861,0.008016,0.008179,0.010452,0.010174


In [ ]:
#Chart showing the difference between students with and without disabilities who were above average
df_zip_ecodis_not_above = alt.Chart(df_zip_ecodis_not).mark_bar().encode(
    alt.X('ZIP_CODE:N', sort=alt.EncodingSortField(field="ZIP_CODE", op="count", order='ascending')),
    y=alt.Y('PercentAboveAverage:Q'),
    color = 'TestingGroup:N',
    tooltip=['ZIP_CODE', 'TestingGroup','PercentAboveAverage']
).interactive(
).properties(
        title = 'Average Percent of Non vs. Economically Disadvantaged Students Above Average Growth per Zip Code'
)

df_zip_ecodis_not_above

alt.Chart(...)

In [ ]:
#Chart showing the difference between students with and without disabilities who were above average
perc_above_avg_eco = alt.Chart(df_zip_ecodis_not).mark_bar().encode(
    x=alt.X('TestingGroup:N', title = None),
    y='PercentAboveAverage:Q',
    color='TestingGroup:N',
    column='ZIP_CODE:N'
)
perc_above_avg_eco

alt.Chart(...)

***Can continue with these visualizations as was done with disabled vs. non-disabled

## Comparing between Certain TestingGroups

In [ ]:
#creating a dataframe to compare testing groups
df_compare_some = new_df.loc[((new_df['TestingGroup'] == 'Students with Disabilities') | (new_df['TestingGroup'] == 'Students without Disabilities') | (new_df['TestingGroup'] == 'Economically Disadvantaged') |
   (new_df['TestingGroup'] == 'Not Economically Disadvantaged') | (new_df['TestingGroup'] == 'Male') | (new_df['TestingGroup'] == 'Female')) & (df_test_int['Grade'] == 'All Grades')]
df_compare_some = df_compare_some.groupby(['ZIP_CODE', 'TestingGroup']).mean()
df_compare_some = df_compare_some.reset_index()
df_compare_some = df_compare_some.sort_values('NumberAboveAverageGrowth', axis = 0, ascending = True)
# df_compare_some.head()

In [ ]:
#creating a dataframe to compare the percent growth of the top 4 most common races in america
american_races_4 = new_df.loc[((new_df['TestingGroup'] == 'Black, not of Hispanic origin') | (new_df['TestingGroup'] == 'Asian') | (new_df['TestingGroup'] == 'White, not of Hispanic origin') | (new_df['TestingGroup'] == 'Hispanic')) & (df_test_int['Grade'] == 'All Grades')]
american_races_4 = american_races_4.groupby(['ZIP_CODE', 'TestingGroup']).mean()
american_races_4 = american_races_4.reset_index()
american_races_4 = american_races_4.sort_values('NumberAboveAverageGrowth', axis = 0, ascending = True)
# american_races_4.head()

In [ ]:
bars = alt.Chart().mark_bar(size = 8).encode(
    x=alt.X('PercentAverageGrowth:Q', stack='zero'),
    y=alt.Y('ZIP_CODE:N'),
    color=alt.Color('TestingGroup')
).properties(
    width = 100,
    height = 300
    # title = 'Percent Avg. Growth in each Zip Code for TestingGroups',
)

text = alt.Chart().mark_text(dx=-17, dy=2, color='white').encode(
    x=alt.X('PercentAverageGrowth:Q', stack='zero'),
    y=alt.Y('ZIP_CODE:N'),
    detail='TestingGroup:N',
    text=alt.Text('PercentAverageGrowth:Q', format='.3f')
)

alt.layer(bars, text, data=df_compare_some).facet(column='TestingGroup:N')


alt.FacetChart(...)

In [ ]:
bars1 = alt.Chart(american_races_4).mark_bar().encode(
    x=alt.X('PercentAverageGrowth:Q', stack='zero'),
    y=alt.Y('ZIP_CODE:N'),
    color=alt.Color('TestingGroup')
).properties(
    width = 100,
    height = 300
)

text1 = alt.Chart(american_races_4).mark_text(dx= -15, dy= 2, color='white').encode(
    x=alt.X('PercentAverageGrowth:Q', stack='zero'),
    y=alt.Y('ZIP_CODE:N'),
    detail='TestingGroup:N',
    text=alt.Text('PercentAverageGrowth:Q', format='.3f')
)


alt.layer(bars1, text1, data=american_races_4).facet(column='TestingGroup:N')


alt.FacetChart(...)

There is not enough data to be able to accurately compare between races for percent average growth per zip code

## Comparing Different School Types

application, magnet, regular

In [ ]:
#creating a sub dataframe to compare between school types
testing_this = new_df.loc[(df_test_int['Grade'] == 'All Grades') & (df_test_int['TestingGroup'] == 'All Students') ]
# testing_this.SCHOOL_EMPHASIS.value_counts()
# # testing_this.shape
# # testing_this

General Education        283
Alternative Education      6
Special Education          4
Name: SCHOOL_EMPHASIS, dtype: int64

In [ ]:
app_schools = ['Academy of The Americas', 'Bates Academy', 'Benjamin Carson High School of Science and Medicine', 'Clippert Academy',
    'Chrysler Elementary School', 'Detroit International Academy', 'Detroit School of Arts',
    'Foreign Language Immersion and Cultural Studies', 'Golightly Education CenterGolightly Education Center',
    'Garvey Academy', 'Paul Robeson Malcolm X Academy', 'The School at Marygrove',
    'Academy of The Americas High School', 'Benjamin Carson High School of Science and Medicine',
    'Davis Aerospace Technical High School at Golightly', 'Detroit International Academy for Young Women',
    'Detroit School of Arts', 'Frederick Douglass Academy for Young Men', 'Brenda Scott Academy for Theatre Arts', 'Edison Elementary School', 'Palmer Park Preparatory Academy',
    'John R. King Academic and Performing Arts Academy', 'Spain Elementary-Middle School']

mag_schools = ['Ann Arbor Trail Magnet School', 'Ludington Magnet Middle School',
    'Fisher Magnet Lower Academy', 'Fisher Magnet Upper Academy']


#getting a list of all the schools that were not included in the magnet or application list
building = list(testing_this.BuildingName)
main_list = list(set(building) - set(app_schools))
reg_schools = list(set(main_list) - set(mag_schools))
reg_schools.sort()
# reg_schools

In [ ]:
#changing the school emphasis to match the results from the list we found on the DPSCD website

testing_this.loc[new_df.BuildingName.isin(app_schools), 'SCHOOL_EMPHASIS'] = 'applied'
testing_this.loc[new_df.BuildingName.isin(mag_schools), 'SCHOOL_EMPHASIS'] = 'magnet'
testing_this.loc[new_df.BuildingName.isin(reg_schools), 'SCHOOL_EMPHASIS'] = 'regular'

testing_this.groupby('SCHOOL_EMPHASIS').sum()

/shared-libs/python3.9/py/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,ISDCode,PhoneNumber,ZIP_CODE,NumberAboveAverageGrowth,NumberAverageGrowth,NumberBelowAverageGrowth,PercentAboveAverage,PercentAverageGrowth,PercentBelowAverage,TotalIncluded,MeanSGP
SCHOOL_EMPHASIS,,,,,,,,,,,
applied,4428,169369338922,2603537,0.896416,0.871062,0.798771,0.460749,0.562591,0.509392,0.891181,0.539692
magnet,1066,40765554516,626823,0.144559,0.205207,0.254806,0.082977,0.118080,0.130735,0.213741,0.103930
regular,18532,708944085596,10897178,3.112494,3.327671,3.406340,1.795565,2.101917,2.131512,3.402376,2.064543


In [ ]:
#creating a data frame to compare all grades for each zipcode
df_emph = testing_this.loc[(testing_this['Grade'] == 'All Grades')]
df_emph = df_emph.groupby(['ZIP_CODE','BuildingName', 'SCHOOL_EMPHASIS']).mean()
df_emph = df_emph.reset_index()
df_emph['counting'] = 1
df_emph = df_emph.groupby(['ZIP_CODE','SCHOOL_EMPHASIS']).sum()
df_emph = df_emph.reset_index()

# df_emph



In [ ]:
#showing the count of each type of school for each zip code 
alt.Chart(df_emph).mark_bar().encode(
    x=alt.X('ZIP_CODE:N', title = None),
    y='counting:Q',
    color = 'SCHOOL_EMPHASIS',
    tooltip = ['SCHOOL_EMPHASIS', 'counting']
).interactive()

alt.Chart(...)

In [ ]:
emphasis = testing_this.loc[(df_test_int['Grade'] == 'All Grades')]
emphasis = emphasis.groupby(['ZIP_CODE', 'SCHOOL_EMPHASIS']).sum()
emphasis = emphasis.reset_index()
# emphasis = emphasis.sort_values('NumberAboveAverageGrowth', axis = 0, ascending = True)


In [ ]:
#seeing which zip codes contain certain types of schools and if there is a 
#noticable difference in percent growth between the school types for precent growth

# perc_above_avg_eco = alt.Chart(emphasis).mark_bar().encode(
#     x=alt.X('SCHOOL_EMPHASIS:N', title = None),
#     y='PercentAboveAverage:Q',
#     color='SCHOOL_EMPHASIS:N',
#     column='ZIP_CODE:N'
# )

perc_at_avg_eco = alt.Chart(emphasis).mark_bar().encode(
    x=alt.X('SCHOOL_EMPHASIS:N', title = None),
    y='PercentAverageGrowth:Q',
    color='SCHOOL_EMPHASIS:N',
      column='ZIP_CODE:N'
)

# perc_below_avg_eco = alt.Chart(emphasis).mark_bar().encode(
#     x=alt.X('SCHOOL_EMPHASIS:N', title = None),
#     y='PercentBelowAverage:Q',
#     color='SCHOOL_EMPHASIS:N',
#     column='ZIP_CODE:N'
# )
perc_at_avg_eco


#No patterns stood out for percent growth between schools

alt.Chart(...)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e115903d-6f9d-4ba6-a598-8111e36ee3b2' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>